# Extract Data from [Football API](https://www.api-football.com/documentation-v3#tag/Players/operation/get-players)

*See player_stats.py for details on functions used below*

## 1. Pull and save all necessary data from API

In [1]:
import player_stats

# Leagues Ids
leagues = [310,
 129,
 131,
 134,
 188,
 218,
 418,
 116,
 144,
 315,
 71,
 72,
 75,
 76,
 172,
 265,
 169,
 239,
 240,
 162,
 210,
 318,
 345,
 119,
 242,
 39,
 40,
 41,
 42,
 45,
 46,
 47,
 48,
 244,
 61,
 62,
 66,
 327,
 78,
 79,
 81,
 197,
 234,
 271,
 164,
 290,
 357,
 383,
 135,
 136,
 137,
 98,
 389,
 371,
 262,
 88,
 407,
 103,
 304,
 250,
 252,
 281,
 106,
 94,
 95,
 305,
 283,
 235,
 179,
 332,
 373,
 292,
 140,
 141,
 143,
 113,
 207,
 203,
 301,
 268,
 270,
 253,
 299,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 9,
 11,
 12,
 13,
 15]

# convert league ids to string
leagues = [str(league) for league in leagues]

print(type(leagues[0]))
print(len(leagues))

<class 'str'>
95


The below code will save json files to the current directory called "seasons.json" and "players.json".

Only call this funciton to get/update the data in the saved files. This allows us to have access to the data locally without making multiple API calls each time we need the data. DO NOT need to run this code if you already have the "players.json" file.

WANING: May take up to 10hrs to run

In [2]:
###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

player_stats.update_files(leagues)

###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

Starting - League: 310 Season: 2008

Page: 1/5 Num players: 20
Page: 2/5 Num players: 20
Page: 3/5 Num players: 20
Page: 4/5 Num players: 20
Page: 5/5 Num players: 3

Starting - League: 310 Season: 2009

Page: 1/12 Num players: 20
Page: 2/12 Num players: 20
Page: 3/12 Num players: 20
Page: 4/12 Num players: 20
Page: 5/12 Num players: 20
Page: 6/12 Num players: 20
Page: 7/12 Num players: 20
Page: 8/12 Num players: 20
Page: 9/12 Num players: 20
Page: 10/12 Num players: 20
Page: 11/12 Num players: 20
Page: 12/12 Num players: 10

Starting - League: 310 Season: 2010

Page: 1/13 Num players: 20
Page: 2/13 Num players: 20
Page: 3/13 Num players: 20
Page: 4/13 Num players: 20
Page: 5/13 Num players: 20
Page: 6/13 Num players: 20
Page: 7/13 Num players: 20
Page: 8/13 Num players: 20
Page: 9/13 Num players: 20
Page: 10/13 Num players: 20
Page: 11/13 Num players: 20
Page: 12/13 Num players: 20
Page: 13/13 Num players: 11

Starting - League: 310 Season: 2011

Page: 1/18 Num players: 20
Page: 2/18 

## 2. Load data from json files

In [3]:
import json

# load seasons
with open("seasons.json", "r") as file:
    seasons = file.readlines()
    seasons = [year.strip("\n") for year in seasons]
    
# load teams
# with open("teams.json", "r") as file:
#     teams = json.loads(file.read())

# load players
with open("players.json", "r") as file:
    players = json.loads(file.read())

print("Num leagues:", len(leagues))
print("Num seasons:", len(seasons))
# print("Num teams per league and season:", len(teams))
print("Num players per league season:", len(players))

Num leagues: 95
Num seasons: 15
Num players per league season: 731584


In [141]:
# look at data format
print(players[0].keys())
print(players[0]['player'].keys())
print(players[0]['statistics'][0].keys())
print()

# example player
print(players[30]['player']['name'])
# one set of statistics per team player belonged to in the season
print(players[30]['statistics'][0])
print(players[30]['statistics'][1])

dict_keys(['player', 'statistics'])
dict_keys(['id', 'name', 'firstname', 'lastname', 'age', 'birth', 'nationality', 'height', 'weight', 'injured', 'photo'])
dict_keys(['team', 'league', 'games', 'substitutes', 'shots', 'goals', 'passes', 'tackles', 'duels', 'dribbles', 'fouls', 'cards', 'penalty'])

Mariglen Kapaj
{'team': {'id': 3325, 'name': 'Bylis', 'logo': 'https://media-3.api-sports.io/football/teams/3325.png'}, 'league': {'id': None, 'name': 'Superliga', 'country': None, 'logo': None, 'flag': None, 'season': '2008-2009'}, 'games': {'appearences': None, 'lineups': None, 'minutes': None, 'number': None, 'position': 'Midfielder', 'rating': None, 'captain': False}, 'substitutes': {'in': None, 'out': None, 'bench': None}, 'shots': {'total': None, 'on': None}, 'goals': {'total': 1, 'conceded': None, 'assists': None, 'saves': None}, 'passes': {'total': None, 'key': None, 'accuracy': None}, 'tackles': {'total': None, 'blocks': None, 'interceptions': None}, 'duels': {'total': None, 'won'

## 3. Convert data to pandas table and export to CSV

In [81]:
import pandas as pd

# flatten the list of players (list of nested dicts) into pandas table
df1 = pd.json_normalize(players, sep="_")
# duplicate rows for each statistic a player has
df1 = df1.explode("statistics").reset_index(drop=True)

# df1 rows are each combination of player, league, team
# df1 cols are all the player info and statistcs (note the statistics column needs to be reformatted)
df1.shape

(762670, 14)

In [82]:
# flatten the list of statistics of player into another pandas table
df2 = pd.json_normalize(df1["statistics"], sep="_").reset_index(drop=True)

# df1 rows are each combination of player, league, team
# df1 cols are all the player statistics
df2.shape

(762670, 46)

In [85]:
# concatenate the player info and player statistics tables into one
final_df = pd.concat([df1.drop(["statistics"], axis=1), df2], axis=1)
print(final_df.shape)
final_df.head()

(762670, 59)


,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
0,4299,Endri Bakiu,Endri,Bakiu,32.0,1987-01-06,Tiranë,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,142562,Pedro Paulo Pinheiro Neves,Pedro Paulo,Pinheiro Neves,39.0,1981-10-06,Lisboa,Portugal,Portugal,178 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,210892,Roland Dervishi,Roland,Dervishi,34.0,1982-02-16,Peqin,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,210924,Gersi Arbri,Gersi,Arbri,33.0,1984-08-01,Berat,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,211075,Klodian Arbri,Klodian,Arbri,37.0,1979-09-10,Berat,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# save dataframe as csv
final_df.to_csv("player_stats.csv", index=False, encoding="utf-8-sig")

## 4. Exploratory Data Analysis

TO-DO: Need to consider which rows to drop (and for which columns / types of players) since there are many NaN

In [103]:
stat_list = final_df.columns.to_list()[22:]
stat_list

['games_appearences',
 'games_lineups',
 'games_minutes',
 'games_number',
 'games_position',
 'games_rating',
 'games_captain',
 'substitutes_in',
 'substitutes_out',
 'substitutes_bench',
 'shots_total',
 'shots_on',
 'goals_total',
 'goals_conceded',
 'goals_assists',
 'goals_saves',
 'passes_total',
 'passes_key',
 'passes_accuracy',
 'tackles_total',
 'tackles_blocks',
 'tackles_interceptions',
 'duels_total',
 'duels_won',
 'dribbles_attempts',
 'dribbles_success',
 'dribbles_past',
 'fouls_drawn',
 'fouls_committed',
 'cards_yellow',
 'cards_yellowred',
 'cards_red',
 'penalty_won',
 'penalty_commited',
 'penalty_scored',
 'penalty_missed',
 'penalty_saved']

In [125]:
# for attackers
attackers = final_df[final_df["games_position"] == "Attacker"]
attackers = attackers[stat_list].isna().sum() / attackers[stat_list].shape[0]
attackers.name = "Attacker"

# for midfielders
midfielders = final_df[final_df["games_position"] == "Midfielder"]
midfielders = midfielders[stat_list].isna().sum() / midfielders[stat_list].shape[0]
midfielders.name = "Midfielder"

# for defenders
defenders = final_df[final_df["games_position"] == "Defender"]
defenders = defenders[stat_list].isna().sum() / defenders[stat_list].shape[0]
defenders.name = "Defender"

# for goalkeepers
goalkeepers = final_df[final_df["games_position"] == "Goalkeeper"]
goalkeepers = goalkeepers[stat_list].isna().sum() / goalkeepers[stat_list].shape[0]
goalkeepers.name = "Goalkeeper"

# total
total = final_df[stat_list].isna().sum() / final_df[stat_list].shape[0]
total.name = "Total"

percent_nan = (100 * pd.concat([attackers, midfielders, defenders, goalkeepers, total], axis=1)).round(2)
percent_nan.sort_values(by=["Total"], ascending=False)

,Attacker,Midfielder,Defender,Goalkeeper,Total
dribbles_past,100.00,100.00,100.00,100.00,100.00
games_number,100.00,100.00,100.00,100.00,100.00
penalty_won,97.79,99.05,99.46,100.00,98.99
penalty_commited,99.64,99.13,97.42,100.00,98.80
goals_saves,100.00,100.00,99.99,84.98,98.39
penalty_saved,100.00,100.00,99.99,78.57,97.71
goals_assists,90.03,90.91,93.37,99.52,92.41
tackles_blocks,87.86,86.49,83.88,94.43,86.83
goals_conceded,86.13,86.47,86.01,78.56,85.41
tackles_total,84.52,83.39,82.64,97.46,84.91


In [139]:
final_df["passes_accuracy"].value_counts(dropna=False)
final_df.dropna(subset=stat_list, how="all")

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
0,4299,Endri Bakiu,Endri,Bakiu,32.0,1987-01-06,Tiranë,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,142562,Pedro Paulo Pinheiro Neves,Pedro Paulo,Pinheiro Neves,39.0,1981-10-06,Lisboa,Portugal,Portugal,178 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,210892,Roland Dervishi,Roland,Dervishi,34.0,1982-02-16,Peqin,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,210924,Gersi Arbri,Gersi,Arbri,33.0,1984-08-01,Berat,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,211075,Klodian Arbri,Klodian,Arbri,37.0,1979-09-10,Berat,Albania,Albania,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762665,30408,Gerson,Gerson,Santos da Silva,26.0,1997-05-20,Rio de Janeiro,Brazil,Brazil,188 cm,...,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
762666,311157,Matheus França,Matheus,França de Oliveira,19.0,2004-04-01,None,Brazil,Brazil,178 cm,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
762667,2414,Everton,Everton,Sousa Soares,27.0,1996-03-22,Fortaleza,Brazil,Brazil,174 cm,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
762668,51107,N. Tolo,Nouhou,Tolo,26.0,1997-06-23,Douala,Cameroon,Cameroon,178 cm,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # for attackers
# attackers = final_df[final_df["games_position"] == "Attacker"]
# attackers = (attackers[stat_list] == 0).astype(int).sum(axis=0) / attackers[stat_list].shape[0]
# attackers.name = "Attacker"

# # for midfielders
# midfielders = final_df[final_df["games_position"] == "Midfielder"]
# midfielders = (midfielders[stat_list] == 0).astype(int).sum(axis=0) / midfielders[stat_list].shape[0]
# midfielders.name = "Midfielder"

# # for defenders
# defenders = final_df[final_df["games_position"] == "Defender"]
# defenders = (defenders[stat_list] == 0).astype(int).sum(axis=0) / defenders[stat_list].shape[0]
# defenders.name = "Defender"

# # for goalkeepers
# goalkeepers = final_df[final_df["games_position"] == "Goalkeeper"]
# goalkeepers = (goalkeepers[stat_list] == 0).astype(int).sum(axis=0) / goalkeepers[stat_list].shape[0]
# goalkeepers.name = "Goalkeeper"

# # total
# total = (final_df[stat_list] == 0).astype(int).sum(axis=0) / final_df[stat_list].shape[0]
# total.name = "Total"

# percent_nan = (100 * pd.concat([attackers, midfielders, defenders, goalkeepers, total], axis=1)).round(2)
# percent_nan.sort_values(by=["Total"], ascending=False)